In [7]:
from spacy import load
model = load('en_core_web_sm')
print('loaded spacy model')

loaded spacy model


In [8]:
import pandas as pd
df = pd.read_csv(filepath_or_buffer='/kaggle/input/inshorts-dataset-english/english_news_dataset.csv', parse_dates=['Date'], dayfirst=True)
print(df.shape)

(43610, 4)


In [9]:
parts = {}
for index, headline in enumerate(df['Headline'].values.tolist()):
    tokens = model(headline)
    for token in tokens:
        if token.text in parts:
            parts[token].append(token.pos_)
        else:
            parts[token] = [token.pos_]
    if index > 0 and index % 10000 == 0:
        print(len(parts))
print(len(parts))

130953
262443
394304
527092
574932


In [20]:
def get_parts(arg: str):
    tokens = model(arg)
    return [(token.text, token.pos_) for token in tokens]
df['headline_parts'] = df['Headline'].apply(get_parts)
df.head()['headline_parts']

0    [(Congress, PROPN), (leader, NOUN), (Baljinder...
1    [(17, NUM), (-, PUNCT), (year, NOUN), (-, PUNC...
2    [(Hampers, NOUN), (to, PART), (welcome, VERB),...
3    [(Only, ADV), (10, NUM), (%, NOUN), (women, NO...
4    [(Ganesh, ADJ), (temple, NOUN), (decorated, VE...
Name: headline_parts, dtype: object

In [ ]:
df['headline'] = df['Headline'].str.lower()
df['headline_lower_parts'] = df['headline'].apply(get_parts)
df.head(n=10)['headline_lower_parts']

In [18]:
from collections import Counter

def get_noun_percentage(counter: dict) -> float:
    result = 0
    if 'NOUN' in counter.keys():
        result += counter['NOUN']
    if 'PROPN' in counter.keys():
        result += counter['PROPN']
    return result / sum(list(counter.values()))        
    
counters = {key: dict(Counter(value)) for key, value in parts.items()}
noun_percentages = {key: get_noun_percentage(value) for key, value in counters.items()}
noun_percent_df = pd.DataFrame(data={'token': list(noun_percentages.keys()), 'percent': list(noun_percentages.values())})
noun_percent_df.sort_values(ascending=False, by='percent')

,token,percent
0,Congress,1.0
308913,alert,1.0
308936,assault,1.0
308933,job,1.0
308930,CEO,1.0
...,...,...
323028,there,0.0
323027,if,0.0
323026,as,0.0
323025,Felt,0.0


In [19]:
noun_percent_df['percent'].value_counts()

percent
0.0    321911
1.0    253021
Name: count, dtype: int64

In [13]:
df[['Headline']].head()

,Headline
0,Congress leader Baljinder Singh shot dead at h...
1,17-year-old girl preparing for NEET dies by su...
2,Hampers to welcome MPs in new Parliament tomor...
3,"Only 10% women lawmakers in RS, while only 14%..."
4,"Ganesh temple decorated with notes, coins wort..."


In [16]:
for index, row in df.head().iterrows():
    tokens = model(row['Headline'])
    for token in tokens:
        print(token.text, token.pos_)

Congress PROPN
leader NOUN
Baljinder PROPN
Singh PROPN
shot VERB
dead ADJ
at ADP
home NOUN
in ADP
Punjab PROPN
's PART
Moga NOUN
17 NUM
- PUNCT
year NOUN
- PUNCT
old ADJ
girl NOUN
preparing VERB
for ADP
NEET PROPN
dies NOUN
by ADP
suicide NOUN
in ADP
Rajasthan PROPN
's PART
Kota PROPN
Hampers NOUN
to PART
welcome VERB
MPs NOUN
in ADP
new ADJ
Parliament PROPN
tomorrow NOUN
, PUNCT
pics NOUN
surface VERB
Only ADV
10 NUM
% NOUN
women NOUN
lawmakers NOUN
in ADP
RS PROPN
, PUNCT
while SCONJ
only ADV
14 NUM
% NOUN
in ADP
LS PROPN
: PUNCT
INC PROPN
's PART
Kharge PROPN
Ganesh ADJ
temple NOUN
decorated VERB
with ADP
notes NOUN
, PUNCT
coins NOUN
worth ADJ
₹ NOUN
2.5 NUM
crore NOUN
in ADP
Bengaluru PROPN
